In [ ]:
#the logic is 
#take the input make the q k v things
# break them down (by dimention of embedding // nb heads)
#apply the fomua of attention thing 
#concat

In [12]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F

class MultiheadAttention(nn.Module):
    def __init__(self,batch_size=1,seq_length=10,nb_heads=8,d_model=512,dropout=0.1):
        super(MultiheadAttention, self).__init__()
        assert d_model% nb_heads==0
        self.d_model = d_model
        self.nb_heads=nb_heads
        self.d_k = d_model // nb_heads
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_forward = nn.Linear(d_model, d_model) #for combind gthe outputs later
        #in here we just defined all we would need we did not
        #resize/rescale anything /mul anything yet we
        # do all that mess in foirward
    def scaled_dot_prod(self,q,v,k,d_k):
        #the fc that does the main logic idea
        #that i know the math but just gonna copy how they write it
        scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
        attention = F.softmax(scaled, dim=-1)
        values = torch.matmul(attention, v)
        return values
    def pass_heads(self,x):
        #now pass that shit through the heads
        #this function changes teh struc of the treatment so
        #it is on multiple heads instead of one
        #"one big attention" to "multiple small heads."
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.nb_heads, self.d_k).transpose(1, 2)
    def combine_heads(self,x):
        batch_size, num_heads, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
    def forward(self,q,k,v):
        Q = self.pass_heads(self.W_q(q))
        K = self.pass_heads(self.W_k(k))
        V = self.pass_heads(self.W_v(v))
        print(Q.shape)
        #what this do is change the dimensions of the q,k,v
     #they are no longer working with single head they have many now
        attn_output = self.scaled_dot_prod(Q, K, V, self.d_k)
        output = self.W_forward(self.combine_heads(attn_output))
        return output

        
        
        
    
        
        
        
        
        
        
        


In [13]:
d_model = 8
num_heads = 4
query_seq_length = 5
ke_seq_length = 6
batch_size = 1

mha = MultiheadAttention(batch_size,query_seq_length, num_heads,d_model)

Q = torch.randn(batch_size, query_seq_length, d_model)
K =  V = torch.randn(batch_size, ke_seq_length, d_model)
 
mask = torch.ones(batch_size, 1, query_seq_length, ke_seq_length)

# Forward pass
output = mha(Q, K, V)

torch.Size([1, 4, 5, 2])


In [11]:
print(output)

tensor([[[-0.1781, -0.2942,  0.2713,  0.1556, -0.0173, -0.1651, -0.3160,
           0.0702],
         [-0.2155, -0.3244,  0.2264,  0.2289,  0.0026, -0.1848, -0.2680,
           0.0952],
         [-0.2071, -0.2984,  0.2440,  0.1849, -0.0024, -0.1802, -0.2867,
           0.0799],
         [-0.1839, -0.2773,  0.2044,  0.2137,  0.0723, -0.1786, -0.3495,
           0.0462],
         [-0.1440, -0.2833,  0.2329,  0.1591,  0.0198, -0.1609, -0.3903,
           0.0553]]], grad_fn=<ViewBackward0>)
